In [ ]:
#hide
#skip
! [ -e /content ] && pip install -Uqq pip gingado nbdev # install or upgrade gingado on colab

# Using gingado to understand economic growth
> An illustration with Barro and Lee (1994)

This notebook showcases one possible use of `gingado` by estimating economic growth across countries, using the dataset studied by Barro and Lee (1994, "Sources of economic growth", Carnegie-Rochester Conference Series on Public Policy, 40, p.1-46). You can run this notebook interactively, by clicking on the appropriate link above.

This dataset has been widely studied in economics. Belloni et al (2011, "Inference for high-dimensional sparse econometric models") and Giannone et al (2021, "Economic predictions with big data: the illusion of sparsity", Econometrica, 89, 5, p.2409-2437) are two studies of this dataset that are most related to machine learning.

This notebook will use `gingado` to compare different classes of models (and their combination in a single model), select the best performing alternative, and document it. Because the notebook is for pedagogical purposes only, please bear in mind some aspects of the machine learning workflow (such as carefully thinking about the cross-validation strategy) are glossed over in this notebook.

For a more thorough description of `gingado`, please refer to the package's [website](https://dkgaraujo.github.io/gingado) and to the academic [material](https://www.github.com/dkgaraujo/gingado_comms) about it.

## Setting the stage

We will import packages as the work progresses. This will help highlight the specific steps in the workflow that `gingado` can be helpful with.

In [ ]:
import pandas as pd
from scipy import io

The data is available in the [online annex](https://onlinelibrary.wiley.com/doi/abs/10.3982/ECTA17842) to Giannone et al (2021). In that paper, this dataset corresponds to what the authors call "macro2".

In [ ]:
growth_data = io.loadmat('data/GrowthData.mat')
colnames = [m[0].strip() for m in growth_data['Mnem'][0]]
df = pd.DataFrame(growth_data['data'], columns=colnames)

FileNotFoundError: [Errno 2] No such file or directory: 'data/CvSamplesMacro2.mat'

The dataset contains explanatory variables and the outcome variable, representing per-capita growth between 1960 and 1985, for 90 countries.

In [ ]:
df

For the remainder of this notebook, the dataset will be divided into an `X` dataset of regressors and a `y` vector of outcomes.

In [ ]:
from sklearn.model_selection import train_test_split

y = df.pop('Outcome')
X = df

In [ ]:
y.plot.hist(bins=90)

In [ ]:
X

# Establishing a benchmark model

Generally speaking, it is a good idea to establish a benchmark model at the first stages of development of the machine learning model. `gingado` offers a class of automatic benchmarks that can be used off-the-shelf depending on the task at hand: `RegressionBenchmark` and `ClassificationBenchmark`. It is also good to keep in mind that more advanced users can create their own benchmark on top of a base class provided by `gingado`: `ggdBenchmark`.

For this application, since we are interested in running a regression task, we will use `RegressionBenchmark`:

In [ ]:
from gingado.benchmark import RegressionBenchmark

What this object does is the following:
* it creates a random forest
* three different vesions of the random forest are trained on the user data
* the version that performs better is chosen as the benchmark
* right after it is trained, the benchmark is documented using `gingado`'s `ModelCard` documenter.

The user can easily change the parameters above. For example, instead of a random forest the user might prefer a neural network as the benchmark. Or, in lieu of the default parameters provided by `gingado`, users might have their own idea of what could be a reasonable parameter space to search.

Random forests are chosen as the go-to benchmark algorithm because of their reasonably good performance in a wide variety of settings, the fact that they don't require much data transformation (ie, normalising the data to have zero mean and one standard deviation), and by virtue of their relatively transparency about the importance of each regressor.

The first step is to initialise the benchmark object. At this time, we pass some arguments about how we want it to behave. In this case, we set the verbosity level to produce output related to each alternative considered.

In [ ]:
benchmark = RegressionBenchmark(verbose_grid=3)

The next step is to fit the `benchmark` object to the data. 

In [ ]:
#collapse_output
benchmark.fit(X, y)

As we can see above, with a few lines we have trained a random forest on the dataset. In this case, the benchmark was the better of three versions of the random forest: one with 50 estimators, another with 100 and the thid with 250 estimators. They were each trained using a 5-fold cross-validation. Let's see which one was the best performing in this case, and hence our benchmark model:

In [ ]:
pd.DataFrame(benchmark.benchmark.cv_results_)

The values above are calculated with $R^2$, the default scoring function for a random forest from the `scikit-learn` package. Suppose that instead we would like a benchmark model that is optimised on the maximum error, ie a benchmark that minimises the worst deviation from prediction to ground truth for all the sample. These are the steps that we would take. Note that a more complete list of ready-made scoring parameters and how to create your own function can be found [here](https://scikit-learn.org/stable/modules/model_evaluation.html#).

In [ ]:
#collapse_output
benchmark_lower_worsterror = RegressionBenchmark(scoring='max_error', verbose_grid=3)
benchmark_lower_worsterror.fit(X, y)

In [ ]:
pd.DataFrame(benchmark_lower_worsterror.benchmark.cv_results_)

Now we even have two benchmark models 😎.

We could further tweek and adjust them, but one of the ideas behind having a benchmark is that it is simple and easy to set up. 

Let's now look at the predictions, comparing them to the original growth values.

In [ ]:
y_pred = benchmark.predict(X)

pd.DataFrame({
    'y': y,
    'y_pred': y_pred
    }).plot.scatter(x='y', y='y_pred', grid=True)

And now a histogram of the benchmark's errors:

In [ ]:
pd.DataFrame(y - y_pred).plot.hist(bins=30)

Since the benchmark is a random forest model, we can see what are the most important regressors, measured as the average reduction in impurity across the trees in the random forest that actually use that particular regressor. They are scaled so that the sum for all features is one. Higher importance amounts indicate that that particular regressor is a more important contributor to the final prediction.

In [ ]:
regressor_importance = pd.DataFrame(benchmark.benchmark.best_estimator_.feature_importances_, index=X.columns, columns=["Importance"])

regressor_importance.sort_values(by="Importance", ascending=False).plot.bar(figsize=(20, 8))

From the graph above, we can see that the regressor `bmp1l` (black-market premium on foreign exchange) predominates. Interestingly, Belloni et al (2011) using squared-root lasso also find this regressor to be important.

## Model documentation

Importantly, the benchmark already has some baseline documentation set up. If the user wishes, they can use that as a basis to document their model. Note that the output is in a raw format that is suitable for machine reading and writing. Intermediary and advanced users may wish to use that format to construct personalised forms, documents, etc.

In [ ]:
#collapse_output
benchmark.model_documentation.show_json()

Since there is some information in the model documentation that was automatically added, we might want to concentrate on the fields in the model card that are yet to be answered. Actually, this is the purpose of `gingado`'s automatic documentation: to afford users more time so they can invest, if they want, on model documentation.

In [ ]:
#collapse_output
benchmark.model_documentation.open_questions()

Let's fill some information:

In [ ]:
benchmark.model_documentation.fill_info({
    'intended_use': {
        'primary_uses': 'This model is trained for pedagogical uses only.',
        'primary_users': 'Everyone is welcome to follow the description showing the development of this benchmark.'
    }
})

Note the format, based on a Python dictionary. In particular, the `open_questions` method results include keys divided by double underscores. As seen above, these should be interpreted as different levels of the documentation template, leading to a nested dictionary. 

Now when we confirm that the questions answered above are no longer "open questions":

In [ ]:
#collapse_output
benchmark.model_documentation.open_questions()

If we want, at any time we can save the documentation to a local JSON file, as well as read another document.

## Trying out model alternatives

The benchmark model may be enough for some analyses, or maybe the user is interested in using the benchmark to explore the data and have an understanding of the importance of each regressor, to concentrate their work on data that can be meaningful for their purposes. But oftentimes a user will want to seek a machine learning model that performs as well as possible.

For users that want to manually create other models, `gingado` allows the possibility of comparing them with the benchmark. If the user model is better, it becomes the new benchmark!

For the following analyses, we will use K-fold as cross-validation, with 5 splits of the sample.

### First candidate: a gradient boosting tree

In [ ]:
#collapse_output
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

param_grid = {
    'learning_rate': [0.01, 0.1, 0.25],
    'max_depth': [3, 6, 9]
}

reg_gradbooster = GradientBoostingRegressor()

gradboosterg_grid = GridSearchCV(
    reg_gradbooster,
    param_grid,
    n_jobs=-1,
    verbose=3
).fit(X, y)

In [ ]:
y_pred = gradboosterg_grid.predict(X)
pd.DataFrame({
    'y': y,
    'y_pred': y_pred
    }).plot.scatter(x='y', y='y_pred', grid=True)

In [ ]:
pd.DataFrame(y - y_pred).plot.hist(bins=30)

### Second candidate: lasso

In [ ]:
#collapse_output
from sklearn.linear_model import Lasso

param_grid = {
    'alpha': [0.5, 1, 1.25],
}

reg_lasso = Lasso(fit_intercept=True)

lasso_grid = GridSearchCV(
    reg_lasso,
    param_grid,
    n_jobs=-1,
    verbose=3
).fit(X, y)

In [ ]:
y_pred = lasso_grid.predict(X)
pd.DataFrame({
    'y': y,
    'y_pred': y_pred
    }).plot.scatter(x='y', y='y_pred', grid=True)

In [ ]:
pd.DataFrame(y - y_pred).plot.hist(bins=30)

## Comparing the models with the benchmark

`gingado` allows users to compare different candidate models with the existing benchmark in a very simple way: using the `compare` method.

In [ ]:
#collapse_output
candidates = [gradboosterg_grid, lasso_grid]
benchmark.compare(X, y, candidates)

The output above clearly indicates that after evaluating the models - and their ensemble together with the existing benchmark - at least one of them was better than the current benchmark. Therefore, it will now be the new benchmark.

In [ ]:
y_pred = benchmark.predict(X)
pd.DataFrame({
    'y': y,
    'y_pred': y_pred
    }).plot.scatter(x='y', y='y_pred', grid=True)

In [ ]:
pd.DataFrame(y - y_pred).plot.hist(bins=30)

## Model documentation

After this process, we can now see how the model documentation was updated automatically:

In [ ]:
#collapse_output
benchmark.model_documentation.show_json()